## Compare a "Composite" and a "Simple" portfolio with the same optimization parameters
### "Composite" portfolio : `[CorrClusterSelector, DualMomentumSelector, CVaRAnalyzer]`
It is a `ModelPipeline` of 2 Selctors and 1 optimizer
### "Simple portfolio" : `Port_CVaR`
Just the optimizer

In [10]:
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
import azapy as az
print(f"azapy version {az.version()} >= 1.2.0", flush=True)

azapy version 1.2.3 >= 1.2.0


### Collect market data

In [11]:
mktdir = '../MkTdata'
sdate = '2012-01-01'
edate = 'today'

symb = ['GLD', 'TLT', 'IHI', 'VHT', 'OIH',
        'XAR', 'XBI', 'XHE', 'XHS', 'XLB',
        'XLE', 'XLF', 'XLI', 'XLK', 'XLU', 
        'XLV', 'XLY', 'XRT', 'SPY', 'ONEQ', 
        'QQQ', 'DIA', 'ILF', 'XSW', 'PGF', 
        'IDV', 'JNK', 'HYG', 'SDIV', 'VIG', 
        'SLV', 'AAPL', 'MSFT', 'AMZN', 'GOOG', 
        'IYT', 'VUG', 'IWM', 'BRK-B', 'ITA']

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir, 
                     verbose=False)

Setup parameters for Selectors: `CorrClusterSelector` and `DualMomentumSelector`,
as well as for `CVaRAnalyzer` optimizer

In [12]:
corr_threshold = 0.98
freq = 'Q'

nw = 5
ths = np.floor(len(symb) * 0.3)

alpha = [0.95, 0.9]
hlength = 1.25
rtype = 'Sharpe'
mu0 = 0.01

histoffset = hlength
fixoffset = -1

### Build and run backtesting for a composed model 
#### `[CorrClusterSelector, DualMomentumSelector, CVaRAnalyzer]`

In [13]:
pname = 'Comoposite'
ccs = az.CorrClusterSelector(corr_threshold=corr_threshold, freq=freq)
dms = az.DualMomentumSelector(nw=nw, threshold=ths)
cvar = az.CVaRAnalyzer(alpha=alpha, freq=freq, hlength=hlength,
                       rtype=rtype, mu0=mu0)
model = az.ModelPipeline([ccs, dms, cvar])
pp = az.Port_Generator(mktdata, 
                       freq=freq, 
                       fixoffset=fixoffset, 
                       histoffset=histoffset,
                       pname=pname)
port = pp.set_model(model)
_ = pp.port_view(fancy=True)

### Run backtesting for simple CVaR model (same parameters and rolling schedule)

In [14]:
pname = 'CVaR'
pp1 = az.Port_CVaR(mktdata, 
                   freq=freq, 
                   histoffset=histoffset, 
                   fixoffset=fixoffset, 
                   pname=pname)
port1 = pp1.set_model(alpha=alpha, rtype=rtype, mu0=mu0, hlength=hlength)
_ = pp1.port_view(fancy=True)

### Compare the two portfolio strategies 

In [15]:
pq = az.Port_Simple([port, port1])
_ = pq.set_model()
_ = pq.port_view_all(componly=True, fancy=True)

### Summary of perfromance in terms of average annual return, maximum drawdown, RoMaD (return over maximum drawdown)

In [16]:
pq.port_perf(componly=True, fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
Comoposite,16.85,-30.83,0.546373,2022-05-24,2022-01-03,2023-02-02,395
CVaR,11.11,-27.26,0.407351,2020-03-23,2020-02-18,2020-07-09,142


### Compare annual performances

In [17]:
(pp.port_annual_returns().join(pp1.port_annual_returns()) * 100).round(2)

,Comoposite,CVaR
year,,
2013,16.93,16.26
2014,-8.46,15.28
2015,10.51,3.82
2016,30.66,-0.55
2017,16.28,27.27
2018,17.95,-4.66
2019,21.75,28.59
2020,46.08,18.16
2021,5.68,11.21


### Compare rebalancing period returns

In [18]:
zzp = pp.port_period_returns().drop('Dfix', axis=1).set_index('Droll')[['RR']].rename({'RR': pp.pname}, axis=1)
zzp1 = pp1.port_period_returns().drop('Dfix', axis=1).set_index('Droll')[['RR']].rename({'RR': pp1.pname}, axis=1)
(zzp.join(zzp1) * 100).round(2)

,Comoposite,CVaR
Droll,,
2013-06-25,9.42,5.37
2013-09-25,6.42,9.00
2013-12-26,2.37,3.17
2014-03-26,0.94,1.32
2014-06-25,1.70,1.60
2014-09-25,-13.15,8.47
2014-12-26,2.47,4.82
2015-03-26,6.05,-3.17
2015-06-25,-4.70,-7.17
